1. Wordcloud

In [17]:
import os
import jieba
import jieba.posseg
import jieba.analyse
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
import numpy as np
import json

# --------------------- 配置区 ---------------------
REPORT_DIR = "Final Project/beijing_gov_reports"  # 存放政府工作报告 TXT 的目录
OUTPUT_DIR = "Final Project/wordclouds"           # 词云输出目录
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 停用词（过滤通用词，保留名词）
STOPWORDS = set([
    '的', '了', '在', '是', '有', '和', '就', '不', '人', '都', '一',
    '上', '也', '很', '到', '说', '要', '去', '会', '着', '没有', '这',
    '北京', '本市', '年', '月', '日', '表示', '强调', '指出', '提出',
    '建设', '推进', '发展', '体系', '改革', '加强', '完善'
])

# 全局词汇-颜色映射（同词同色，不同词不同色）
word_color_map = {}
# 定义更丰富的基础配色，参考目标词云色彩风格，包含红、橙、黄、绿、蓝、紫等色系
base_colors = [
    '#C62828', '#EF6C00', '#FDD835', '#7CB342', '#1565C0', 
    '#6A1B9A', '#455A64', '#FF8A80', '#8E24AA', '#00ACC1'
]

# --------------------- 工具函数 ---------------------
def read_report(file_path):
    """读取政府工作报告文本"""
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def preprocess_text(text, year):
    """预处理文本：过滤非名词 + TF-IDF 提取特色词"""
    # 1. 过滤非中文
    text = re.sub(r'[^\u4e00-\u9fa5]', ' ', text)
    
    # 2. 分词：只保留名词（n）和专有名词（nr）
    words = jieba.posseg.cut(text)
    # 动态停用词（补充年份相关通用词）
    dynamic_stopwords = STOPWORDS.union({str(year), f"{year}年"})
    filtered_words = [
        w.word for w in words 
        if w.word.strip() 
        and w.word not in dynamic_stopwords 
        and (w.flag.startswith('n') or w.flag == 'nr')  # 只保留名词
    ]
    
    # 3. TF-IDF 提取年度特色名词（强化名词权重）
    keywords = jieba.analyse.extract_tags(
        ' '.join(filtered_words), 
        topK=200, 
        withWeight=True,
        allowPOS=('n', 'nr')  # 强制保留名词
    )
    
    # 4. 构造词频字典（用于词云：出现次数越多，字号越大）
    word_freq = {}
    for word, weight in keywords:
        # 年度专属词权重翻倍
        if str(year) in word:
            weight *= 2  
        # 转换为整数频次（避免浮点数问题）
        word_freq[word] = int(weight * 1000)  # 放大倍数，让字号差异更明显
    
    return word_freq

def generate_wordcloud(word_freq, year):
    """生成词云（同词同色、不同词不同色 + 英文题注 + 无掩码）"""
    global word_color_map
    
    # 1. 同词同色、不同词不同色逻辑：确保相同词汇颜色一致，不同词汇颜色不同
    def color_func(word, **kwargs):
        if word not in word_color_map:
            # 循环取色（保证不同词不同色，同词同色）
            color_idx = len(word_color_map) % len(base_colors)
            word_color_map[word] = base_colors[color_idx]
        return word_color_map[word]
    
    # 2. 配置词云（无掩码，靠词频和词性突出重点）
    wc = WordCloud(
        font_path='/System/Library/Fonts/PingFang.ttc',  # macOS 字体
        # font_path='C:/Windows/Fonts/simhei.ttf',       # Windows 字体
        width=800, 
        height=600,
        background_color='white',
        color_func=color_func,
        max_words=300,
        stopwords=STOPWORDS,
        prefer_horizontal=0.9  # 让名词更水平显示，提升可读性
    ).generate_from_frequencies(word_freq)
    
    # 3. 保存词云 + 英文题注
    output_path = os.path.join(OUTPUT_DIR, f'wordcloud_{year}.png')
    plt.figure(figsize=(10, 8))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    # 英文题注：Year 202X Beijing Government Work Report Word Cloud
    plt.title(f'Year {year} Beijing Government Work Report Word Cloud', 
              fontsize=16, fontfamily='Arial')
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✅ 生成 {year} 年词云：{output_path}")

def save_all_word_data(all_word_data):
    """保存所有年份的词频字典和颜色信息到一个JSON文件"""
    output_path = os.path.join(OUTPUT_DIR, 'all_years_word_data.json')
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(all_word_data, f, ensure_ascii=False, indent=2)
    
    print(f"✅ 保存所有年份词频和颜色数据：{output_path}")

# --------------------- 主逻辑 ---------------------
def main():
    # 1. 获取报告文件
    report_files = [
        f for f in os.listdir(REPORT_DIR) 
        if f.endswith(".txt") and "政府工作报告" in f
    ]
    if not report_files:
        print(f"❌ 错误：{REPORT_DIR} 目录下无有效报告文件")
        return
    
    all_word_data = []
    
    # 2. 逐个生成词云并收集数据
    for file in report_files:
        # 提取年份
        year_match = re.search(r'(\d{4})', file)
        if not year_match:
            print(f"❌ 跳过 {file}：无有效年份")
            continue
        year = int(year_match.group(1))
        
        # 处理文本：生成词频字典（突出名词）
        file_path = os.path.join(REPORT_DIR, file)
        raw_text = read_report(file_path)
        word_freq = preprocess_text(raw_text, year)
        
        # 生成词云
        if word_freq:
            generate_wordcloud(word_freq, year)
            
            # 收集当前年份的数据
            year_data = {
                'year': year,
                'word_frequencies': word_freq,
                'word_colors': {word: word_color_map.get(word) for word in word_freq}
            }
            all_word_data.append(year_data)
        else:
            print(f"❌ 跳过 {file}：无有效名词")
    
    # 3. 保存所有年份的数据到一个JSON文件
    if all_word_data:
        save_all_word_data(all_word_data)

if __name__ == "__main__":
    main()    

✅ 生成 2023 年词云：Final Project/wordclouds/wordcloud_2023.png
✅ 生成 2024 年词云：Final Project/wordclouds/wordcloud_2024.png
✅ 生成 2021 年词云：Final Project/wordclouds/wordcloud_2021.png
✅ 生成 2025 年词云：Final Project/wordclouds/wordcloud_2025.png
✅ 生成 2022 年词云：Final Project/wordclouds/wordcloud_2022.png
✅ 保存所有年份词频和颜色数据：Final Project/wordclouds/all_years_word_data.json


2. Heatmap

In [2]:
import os
import matplotlib.pyplot as plt
import re
import numpy as np
import json
from matplotlib.font_manager import FontProperties
import matplotlib.colors as mcolors

# --------------------- 配置区 ---------------------
JSON_PATH = "wordclouds/all_years_word_data.json"  # JSON数据文件路径
CHART_DIR = "charts"                              # 图表输出目录
os.makedirs(CHART_DIR, exist_ok=True)

# 中英文关键词映射（复用原有配置）
keyword_translation = {
    '全面': 'Comprehensive',
    '高质量': 'High-Quality',
    '国际': 'International',
    '中心': 'Center',
    '文化': 'Culture',
    '科技': 'Technology',
    '重点': 'Focus',
    '着力': 'Effort',
    '京津冀': 'Beijing-Tianjin-Hebei',
    '机制': 'Mechanism',
    '示范区': 'Demonstration Zone',
    '产业': 'Industry',
    '领域': 'Field',
    '政府': 'Government',
    '协同': 'Collaboration',
    '行动计划': 'Action Plan',
    '国家': 'National',
    '养老': 'Pension',
    '生态': 'Ecology',
    '企业': 'Enterprise',
    '数字': 'Digital',
    '绿色': 'Green',
    '整治': 'Renovation',
    '规划': 'Planning',
    '功能': 'Function',
    '人民': 'People',
    '全市': 'Citywide',
    '水平': 'Level',
    '农村': 'Rural',
    '经济': 'Economy',
    '政策': 'Policy',
    '任务': 'Task',
    '高水平': 'High-Level',
    '战略': 'Strategy',
    '群众': 'Masses',
    '全国': 'National',
    '专项': 'Special',
    '疫情': 'Epidemic',
    '民生': 'Livelihood',
    '社会主义': 'Socialism',
    '地区': 'Region',
    '环境': 'Environment',
    '乡村': 'Rural',
    '时代': 'Era',
    '特色': 'Characteristic',
    '精神': 'Spirit',
    '人才': 'Talent',
    '基层': 'Grassroots',
    '大力': 'Vigorously',
    '智慧': 'Wisdom'
}

# --------------------- 热力图生成函数 ---------------------
def generate_heatmap(all_year_data):
    """生成关键词热力图（带中英文标注、年份倾斜显示）"""
    # 整理数据：取所有年份中出现过的Top50关键词
    all_words = {}
    for year, data in all_year_data.items():
        for word, freq in data['word_freq'].items():
            all_words[word] = all_words.get(word, 0) + freq
    top_words = [w for w, _ in sorted(all_words.items(), key=lambda x: x[1], reverse=True)[:50]]
    
    # 构建年份-关键词矩阵
    years = sorted(all_year_data.keys())
    heatmap_data = []
    for word in top_words:
        row = [all_year_data[year]['word_freq'].get(word, 0) for year in years]
        heatmap_data.append(row)
    
    # 绘图
    plt.figure(figsize=(15, 10))
    # 字体配置（兼容不同系统）
    try:
        font = FontProperties(fname='/System/Library/Fonts/PingFang.ttc')  # macOS 字体
    except:
        font = FontProperties(fname='C:/Windows/Fonts/simhei.ttf')       # Windows 字体
    
    # 绘制热力图（暖色调表示频率高低）
    im = plt.imshow(heatmap_data, cmap='YlOrRd')  
    
    # 坐标轴配置（中英文标注 + 年份倾斜）
    plt.xticks(
        range(len(years)), 
        [str(year) for year in years], 
        fontsize=10, 
        rotation=45,  # 年份倾斜45度避免重叠
        ha='right', 
        fontfamily='Arial'
    )
    plt.yticks(
        range(len(top_words)), 
        [f'{word} ({keyword_translation.get(word, word)})' for word in top_words], 
        fontproperties=font, 
        fontsize=8
    )
    
    # 颜色条（表示频率刻度）
    cbar = plt.colorbar(im)
    cbar.set_label('Frequency (Scaled)', fontfamily='Arial')
    
    # 标题
    plt.title('Keyword Frequency Heatmap Across Years', fontsize=16, fontfamily='Arial')
    plt.tight_layout()
    
    # 保存图表
    output_path = os.path.join(CHART_DIR, 'keyword_heatmap.png')
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✅ 生成关键词热力图：{output_path}")

# --------------------- 数据读取函数 ---------------------
def load_data_from_json(json_path):
    """从JSON文件加载数据并转换为年份映射格式"""
    if not os.path.exists(json_path):
        print(f"❌ 错误：JSON文件不存在 - {json_path}")
        return None
    
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)
        
        # 转换为 {年份: {'word_freq': {...}}} 格式
        year_data = {}
        for item in json_data:
            year = item['year']
            year_data[year] = {
                'word_freq': item['word_frequencies']  # 对应JSON中的词频字段
            }
        return year_data
    
    except json.JSONDecodeError:
        print(f"❌ 错误：JSON文件解析失败 - {json_path}")
        return None
    except KeyError as e:
        print(f"❌ 错误：JSON文件格式不正确，缺少字段 {e}")
        return None

# --------------------- 主逻辑 ---------------------
def main():
    # 从JSON文件加载数据
    all_year_data = load_data_from_json(JSON_PATH)
    if not all_year_data:
        return
    
    # 生成热力图（至少2个年份才生成）
    if len(all_year_data) >= 2:
        generate_heatmap(all_year_data)
    else:
        print("❌ 年份数量不足，无法生成热力图（至少需要2个年份）")

if __name__ == "__main__":
    main()

✅ 生成关键词热力图：charts/keyword_heatmap.png


2. Grouped_bar

In [4]:
import os
import matplotlib.pyplot as plt
import re
import numpy as np
import pandas as pd
from matplotlib.font_manager import FontProperties
import json

# --------------------- 配置区（修改路径为相对路径） ---------------------
JSON_PATH = "wordclouds/all_years_word_data.json"  # JSON数据文件路径
OUTPUT_DIR = "wordclouds"           # 词云输出目录（用于复用颜色映射）
CHART_DIR = "charts"                # 图表输出目录
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(CHART_DIR, exist_ok=True)

# 全局词汇-颜色映射（复用词云生成的颜色）
word_color_map = {}
# 基础配色（与原词云代码保持一致）
base_colors = [
    '#C62828', '#EF6C00', '#FDD835', '#7CB342', '#1565C0', 
    '#6A1B9A', '#455A64', '#FF8A80', '#8E24AA', '#00ACC1'
]

# 中英文关键词映射（用于图表标注）
keyword_translation = {
    '全面': 'Comprehensive',
    '高质量': 'High-Quality',
    '国际': 'International',
    '中心': 'Center',
    '文化': 'Culture',
    '科技': 'Technology',
    '重点': 'Focus',
    '着力': 'Effort',
    '京津冀': 'Beijing-Tianjin-Hebei',
    '机制': 'Mechanism',
    '示范区': 'Demonstration Zone',
    '产业': 'Industry',
    '领域': 'Field',
    '政府': 'Government',
    '协同': 'Collaboration',
    '行动计划': 'Action Plan',
    '国家': 'National',
    '养老': 'Pension',
    '生态': 'Ecology',
    '企业': 'Enterprise',
    '数字': 'Digital',
    '绿色': 'Green',
    '整治': 'Renovation',
    '规划': 'Planning',
    '功能': 'Function',
    '人民': 'People',
    '全市': 'Citywide',
    '水平': 'Level',
    '农村': 'Rural',
    '经济': 'Economy',
    '政策': 'Policy',
    '任务': 'Task',
    '高水平': 'High-Level',
    '战略': 'Strategy',
    '群众': 'Masses',
    '全国': 'National',
    '专项': 'Special',
    '疫情': 'Epidemic',
    '民生': 'Livelihood',
    '社会主义': 'Socialism',
    '地区': 'Region',
    '环境': 'Environment',
    '乡村': 'Rural',
    '时代': 'Era',
    '特色': 'Characteristic',
    '精神': 'Spirit',
    '人才': 'Talent',
    '基层': 'Grassroots',
    '大力': 'Vigorously',
    '智慧': 'Wisdom'
}

# --------------------- 数据读取函数 ---------------------
def load_data_from_json(json_path):
    """从JSON文件加载数据并重建颜色映射"""
    if not os.path.exists(json_path):
        print(f"❌ 错误：JSON文件不存在 - {json_path}")
        return None
    
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)
        
        # 转换为 {年份: {'word_freq': {...}}} 格式，并重建颜色映射
        year_data = {}
        global word_color_map
        word_color_map = {}  # 重置颜色映射
        
        for item in json_data:
            year = item['year']
            year_data[year] = {
                'word_freq': item['word_frequencies']
            }
            # 从JSON中恢复颜色映射（确保与词云颜色一致）
            for word, color in item['word_colors'].items():
                if word not in word_color_map:
                    word_color_map[word] = color
        
        return year_data
    
    except json.JSONDecodeError:
        print(f"❌ 错误：JSON文件解析失败 - {json_path}")
        return None
    except KeyError as e:
        print(f"❌ 错误：JSON文件格式不正确，缺少字段 {e}")
        return None

# --------------------- 图表生成函数 ---------------------
def generate_grouped_bar(all_year_data, top_n=15):
    """生成多年份分组柱状图（复用词云颜色 + 中文正确显示 + 数值标签）"""
    print("生成汇总柱状图...")
    years = sorted(all_year_data.keys())
    if len(years) < 2:
        print("❌ 年份不足，无法生成分组柱状图")
        return
    
    # 提取各年份Top关键词并去重，按总频率排序
    union_words = set()
    for year in years:
        top_words = [w for w, _ in sorted(
            all_year_data[year]['word_freq'].items(), 
            key=lambda x: x[1], reverse=True
        )[:top_n]]
        union_words.update(top_words)
    # 按跨年份总频率排序，取前N个关键词
    union_words = sorted(union_words, key=lambda x: sum(
        all_year_data[year]['word_freq'].get(x, 0) for year in years
    ), reverse=True)[:top_n]
    
    # 构建数据
    freq_data = {word: [all_year_data[year]['word_freq'].get(word, 0) for year in years] for word in union_words}
    df = pd.DataFrame(freq_data, index=years)
    
    # 绘图：重点解决中文显示问题
    plt.figure(figsize=(16, 8))
    # 指定中文字体路径（macOS/Windows 按需切换）
    try:
        font = FontProperties(fname='/System/Library/Fonts/PingFang.ttc')  # macOS
    except:
        font = FontProperties(fname='C:/Windows/Fonts/simhei.ttf')       # Windows
    
    # 复用词云颜色（与词云保持一致）
    colors = [word_color_map.get(word, '#999999') for word in union_words]
    ax = df.plot(kind='bar', width=0.8, color=colors, ax=plt.gca())
    
    # 添加数值标签（支持中文）
    for p in ax.patches:
        height = p.get_height()
        if height > 0:  # 只显示有值的标签
            ax.annotate(f'{height}', 
                        xy=(p.get_x() + p.get_width()/2, height), 
                        xytext=(0, 3), 
                        textcoords='offset points',
                        ha='center', va='bottom', fontproperties=font, fontsize=8)
    
    # 坐标轴与图例配置（中英文对照 + 中文正常显示）
    plt.xticks(rotation=45, ha='right', fontfamily='Arial')  # X轴年份用Arial
    plt.yticks(fontproperties=font)  # Y轴刻度用中文字体
    ax.set_xlabel('Years', fontfamily='Arial', fontsize=12)
    ax.set_ylabel('Frequency (Scaled)', fontfamily='Arial', fontsize=12)
    ax.set_title('Keyword Frequency Comparison Across Years', fontfamily='Arial', fontsize=16)
    
    # 图例：中文关键词 + 英文翻译（放置在右侧避免重叠）
    legend_labels = [f'{word} ({keyword_translation.get(word, word)})' for word in union_words]
    ax.legend(title='Keywords', labels=legend_labels, 
              bbox_to_anchor=(1.05, 1), loc='upper left', 
              prop=font, fontsize=10)
    
    plt.tight_layout()
    
    # 保存图表
    output_path = os.path.join(CHART_DIR, 'grouped_bar.png')
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✅ 生成汇总柱状图：{output_path}")

# --------------------- 主逻辑 ---------------------
def main():
    # 从JSON文件加载数据
    all_year_data = load_data_from_json(JSON_PATH)
    if not all_year_data:
        return
    
    # 生成汇总柱状图（至少2个年份才生成）
    if len(all_year_data) >= 2:
        generate_grouped_bar(all_year_data)
    else:
        print("❌ 年份数量不足，无法生成柱状图（至少需要2个年份）")

if __name__ == "__main__":
    main()

生成汇总柱状图...
✅ 生成汇总柱状图：charts/grouped_bar.png


3. Stream diagram

In [10]:
import os
import json
import plotly.graph_objects as go
from collections import defaultdict

# --------------------- 配置区 ---------------------
JSON_PATH = "wordclouds/all_years_word_data.json"  # 词频与颜色数据
OUTPUT_DIR = "charts"                            # 河流图输出目录
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 显示的关键词数量（控制河流图复杂度）
TOP_N = 20  
# 流量计算方式：'next'（用后一年频率）| 'mean'（两年均值）
FLOW_METHOD = 'mean'  


# --------------------- 辅助函数 ---------------------
def hex_to_rgba(hex_color, alpha=0.5):
    """将十六进制颜色转换为带透明度的rgba格式"""
    hex_color = hex_color.lstrip('#')
    r = int(hex_color[0:2], 16)
    g = int(hex_color[2:4], 16)
    b = int(hex_color[4:6], 16)
    return f'rgba({r}, {g}, {b}, {alpha})'


# --------------------- 数据处理函数 ---------------------
def load_data():
    """加载JSON数据并提取词频、颜色映射和年份列表"""
    if not os.path.exists(JSON_PATH):
        raise FileNotFoundError(f"JSON文件不存在：{JSON_PATH}")
    
    with open(JSON_PATH, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 整理为 {年份: {词: 频率}} 格式
    year_freq = {}
    word_color = {}  # 全局词-颜色映射
    for item in data:
        year = item['year']
        year_freq[year] = item['word_frequencies']
        # 提取颜色映射（确保同词同色）
        for word, color in item['word_colors'].items():
            if word not in word_color:
                word_color[word] = color
    
    # 按年份排序
    sorted_years = sorted(year_freq.keys())
    return year_freq, word_color, sorted_years


def prepare_stream_data(year_freq, word_color, sorted_years):
    """准备河流图所需的数据流"""
    # 1. 筛选高频关键词（跨年份总频率前TOP_N）
    word_total = defaultdict(int)
    for year, freq in year_freq.items():
        for word, count in freq.items():
            word_total[word] += count
    top_words = [w for w, _ in sorted(word_total.items(), key=lambda x: x[1], reverse=True)[:TOP_N]]
    
    # 2. 构建数据流
    stream_data = []
    for word in top_words:
        values = []
        for year in sorted_years:
            values.append(year_freq[year].get(word, 0))
        stream_data.append({
            "name": word,
            "values": values,
            "color": word_color.get(word, "gray")  # 复用词云颜色
        })
    
    return stream_data, sorted_years


# --------------------- 河流图绘制函数 ---------------------
def generate_stream(stream_data, sorted_years):
    """生成河流图（含交互式标签、年份标注、统一色彩）"""
    # 提取数据
    labels = [data["name"] for data in stream_data]
    colors = [data["color"] for data in stream_data]
    values = [data["values"] for data in stream_data]
    
    # 创建河流图数据
    stream_data = go.Figure()
    for i in range(len(labels)):
        stream_data.add_trace(go.Scatter(
            x=sorted_years,
            y=values[i],
            stackgroup="one",  # 堆叠在同一组
            groupnorm="percent",  # 百分比堆叠
            name=labels[i],
            line=dict(color=colors[i], width=0.5),
            fillcolor=hex_to_rgba(colors[i], 0.5)  # 填充颜色带透明度
        ))
    
    # 设置布局（标题、尺寸等）
    stream_data.update_layout(
        title=dict(
            text=f"Keyword Frequency Stream ({sorted_years[0]} to {sorted_years[-1]})",
            font=dict(family="Arial", size=16)
        ),
        width=1000,
        height=800,
        xaxis=dict(
            title="Year",
            tickvals=sorted_years,
            ticktext=[str(year) for year in sorted_years]
        ),
        yaxis=dict(
            title="Percentage of Total Frequency",
            ticksuffix="%"
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    # 生成图表并保存
    output_path = os.path.join(OUTPUT_DIR, "keyword_stream.html")
    stream_data.write_html(output_path)
    print(f"✅ 河流图已保存至：{output_path}")


# --------------------- 主逻辑 ---------------------
def main():
    try:
        # 加载数据
        year_freq, word_color, sorted_years = load_data()
        if len(sorted_years) < 2:
            print("❌ 年份不足（至少需要2年数据），无法生成河流图")
            return
        
        # 准备河流图数据
        stream_data, sorted_years = prepare_stream_data(year_freq, word_color, sorted_years)
        if not stream_data:
            print("❌ 未找到有效数据，无法生成河流图")
            return
        
        # 生成河流图
        generate_stream(stream_data, sorted_years)
    
    except Exception as e:
        print(f"❌ 错误：{str(e)}")


if __name__ == "__main__":
    main()

✅ 河流图已保存至：charts/keyword_stream.html


4. Sankey diagram

In [8]:
import os
import json
import plotly.graph_objects as go
from collections import defaultdict

# --------------------- 配置区 ---------------------
JSON_PATH = "wordclouds/all_years_word_data.json"  # 词频与颜色数据
OUTPUT_DIR = "charts"                            # 桑基图输出目录
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 显示的关键词数量（控制桑基图复杂度）
TOP_N = 20  
# 流量计算方式：'next'（用后一年频率）| 'mean'（两年均值）
FLOW_METHOD = 'mean'  


# --------------------- 辅助函数 ---------------------
def hex_to_rgba(hex_color, alpha=0.5):
    """将十六进制颜色转换为带透明度的rgba格式"""
    hex_color = hex_color.lstrip('#')
    r = int(hex_color[0:2], 16)
    g = int(hex_color[2:4], 16)
    b = int(hex_color[4:6], 16)
    return f'rgba({r}, {g}, {b}, {alpha})'


# --------------------- 数据处理函数 ---------------------
def load_data():
    """加载JSON数据并提取词频、颜色映射和年份列表"""
    if not os.path.exists(JSON_PATH):
        raise FileNotFoundError(f"JSON文件不存在：{JSON_PATH}")
    
    with open(JSON_PATH, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 整理为 {年份: {词: 频率}} 格式
    year_freq = {}
    word_color = {}  # 全局词-颜色映射
    for item in data:
        year = item['year']
        year_freq[year] = item['word_frequencies']
        # 提取颜色映射（确保同词同色）
        for word, color in item['word_colors'].items():
            if word not in word_color:
                word_color[word] = color
    
    # 按年份排序
    sorted_years = sorted(year_freq.keys())
    return year_freq, word_color, sorted_years


def prepare_sankey_data(year_freq, word_color, sorted_years):
    """准备桑基图所需的节点（nodes）和链接（links）数据"""
    # 1. 筛选高频关键词（跨年份总频率前TOP_N）
    word_total = defaultdict(int)
    for year, freq in year_freq.items():
        for word, count in freq.items():
            word_total[word] += count
    top_words = [w for w, _ in sorted(word_total.items(), key=lambda x: x[1], reverse=True)[:TOP_N]]
    
    # 2. 构建节点：每个关键词为一个节点（含颜色）
    nodes = []
    word_id = {word: i for i, word in enumerate(top_words)}  # 关键词到id的映射
    for word in top_words:
        nodes.append({
            "name": word,
            "color": word_color.get(word, "gray")  # 复用词云颜色
        })
    
    # 3. 构建链接：相邻年份同一关键词的频率流动
    links = []
    for i in range(len(sorted_years) - 1):
        year1 = sorted_years[i]
        year2 = sorted_years[i + 1]
        freq1 = year_freq[year1]
        freq2 = year_freq[year2]
        
        # 只处理TOP关键词之间的链接
        for word in top_words:
            count1 = freq1.get(word, 0)
            count2 = freq2.get(word, 0)
            if count1 == 0 or count2 == 0:
                continue  # 跳过无频率的关键词
            
            # 计算流量值（根据设定的方法）
            if FLOW_METHOD == 'mean':
                value = int((count1 + count2) / 2)
            else:  # 'next'
                value = count2
            
            # 转换颜色格式为rgba（解决Plotly不支持8位十六进制的问题）
            link_color = hex_to_rgba(word_color.get(word, '#999999'), 0.5)
            
            # 添加链接（源为year1的词id，目标为year2的词id）
            links.append({
                "source": word_id[word],
                "target": word_id[word],  # 同一关键词流动
                "value": value,
                "color": link_color
            })
    
    return nodes, links, top_words


# --------------------- 桑基图绘制函数 ---------------------
def generate_sankey(nodes, links, top_words, sorted_years):
    """生成桑基图（含交互式标签、年份标注、统一色彩）"""
    # 提取节点颜色和名称
    node_colors = [node["color"] for node in nodes]
    node_labels = [node["name"] for node in nodes]
    
    # 提取链接数据
    source = [link["source"] for link in links]
    target = [link["target"] for link in links]
    value = [link["value"] for link in links]
    link_colors = [link["color"] for link in links]
    
    # 创建桑基图数据
    sankey_data = go.Sankey(
        node=dict(
            pad=15,  # 节点间距
            thickness=20,  # 节点厚度
            line=dict(color="black", width=0.5),  # 节点边框
            color=node_colors,
            label=node_labels
        ),
        link=dict(
            source=source,
            target=target,
            value=value,
            color=link_colors  # 使用rgba格式的颜色
        )
    )
    
    # 设置布局（标题、尺寸等）
    layout = go.Layout(
        title=dict(
            text=f"Keyword Frequency Flow ({sorted_years[0]} to {sorted_years[-1]})",
            font=dict(family="Arial", size=16)
        ),
        width=1000,
        height=800,
        margin=dict(l=50, r=50, t=100, b=50)
    )
    
    # 生成图表并保存
    fig = go.Figure(data=[sankey_data], layout=layout)
    output_path = os.path.join(OUTPUT_DIR, "keyword_sankey1.html")
    fig.write_html(output_path)
    print(f"✅ 桑基图已保存至：{output_path}")


# --------------------- 主逻辑 ---------------------
def main():
    try:
        # 加载数据
        year_freq, word_color, sorted_years = load_data()
        if len(sorted_years) < 2:
            print("❌ 年份不足（至少需要2年数据），无法生成桑基图")
            return
        
        # 准备桑基图数据
        nodes, links, top_words = prepare_sankey_data(year_freq, word_color, sorted_years)
        if not links:
            print("❌ 未找到有效链接数据，无法生成桑基图")
            return
        
        # 生成桑基图
        generate_sankey(nodes, links, top_words, sorted_years)
    
    except Exception as e:
        print(f"❌ 错误：{str(e)}")


if __name__ == "__main__":
    main()

✅ 桑基图已保存至：charts/keyword_sankey.html
